In [ ]:
!pip install japanize-matplotlib

In [ ]:
!pip install gspread

In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline
import japanize_matplotlib
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
japanize_matplotlib.japanize()

In [ ]:
#ドライブへのアクセス
from google.colab import drive
drive.mount('/content/drive')
%cd "myPath"

# データのダウンロード

In [ ]:
cols = [
    'timestamp',
    'mail_adress',
    'sex',
    'friends_in_LINE',
    'GPA',
    'start_day',
    'number_of_companies', # 受けた企業数
    'number_of_offer',     # 内定数
    'best_offer',          # 第一志望群からの内定可否
    'Cooperativeness',     # 協調性
    'Aggressiveness',      # 積極性
    'Action',              # 行動力
    'Communication',       # コミュ力
    'Problem-solving ability',
    'Challange',
    'Job_hunting_action'
    ]

In [ ]:
# 任意のライブラリのインポート
from google.colab import auth
from google.auth import default
from oauth2client.client import GoogleCredentials
import gspread
import pandas as pd

# GSS読み取りのための準備
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# 読み込み対象のスプレットシートのURLの定義
url = 'myPath'

# データをpandasに乗せる
worksheet = gc.open_by_url(url).get_worksheet(0)
data = pd.DataFrame(worksheet.get_all_values()[1:], columns = cols)

In [ ]:
#はいを1に、いいえ, 性別を0に変換
data['best_offer'].replace({'はい':1, 'いいえ':0}, inplace=True)
data['sex'].replace({'男性':1, '女性':0}, inplace=True)
data.head()

In [ ]:
# 使用カラムを限定
use_cols= ['Cooperativeness',
           'Aggressiveness',
           'Action',
           'Communication',
           'Problem-solving ability',
           'Challange',
           'number_of_companies',
           'number_of_offer',
           'best_offer'
           ]

df = data[use_cols]

In [ ]:
df.head()

,Cooperativeness,Aggressiveness,Action,Communication,Problem-solving ability,Challange,number_of_companies,number_of_offer,best_offer
0,3,5,5,5,5,5,12,4,1
1,4,4,2,3,4,2,2,0,0
2,4,3,2,3,4,3,6,4,1
3,3,3,4,3,4,4,10,5,1
4,4,4,5,4,3,4,10,6,1


# 重回帰分析

In [ ]:
#重回帰分析のデータセットじゅんび
X = df.drop(['number_of_companies',	'number_of_offer', 'best_offer'], axis=1)
y_reg = df['number_of_offer']

In [ ]:
import statsmodels.api as sm
model = sm.OLS(y_reg.astype(float), X.astype(float))  # モデルを作成

result = model.fit()      # モデルを適応
print(result.summary())   # 結果を確認

                                 OLS Regression Results                                
Dep. Variable:        number_of_offer   R-squared (uncentered):                   0.749
Model:                            OLS   Adj. R-squared (uncentered):              0.687
Method:                 Least Squares   F-statistic:                              11.96
Date:                Fri, 14 Oct 2022   Prob (F-statistic):                    3.31e-06
Time:                        14:12:24   Log-Likelihood:                         -62.632
No. Observations:                  30   AIC:                                      137.3
Df Residuals:                      24   BIC:                                      145.7
Df Model:                           6                                                  
Covariance Type:            nonrobust                                                  
                              coef    std err          t      P>|t|      [0.025      0.975]
----------------------------

# ロジスティク回帰

In [ ]:
#ロジスティック回帰分析
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm

y_cla = df['best_offer']

model = sm.Logit(y_cla.astype(float), X.astype(float))
result = model.fit(disp=0)
print(result.summary())

                           Logit Regression Results                           
Dep. Variable:             best_offer   No. Observations:                   30
Model:                          Logit   Df Residuals:                       24
Method:                           MLE   Df Model:                            5
Date:                Fri, 14 Oct 2022   Pseudo R-squ.:                  0.3381
Time:                        14:10:03   Log-Likelihood:                -6.4552
converged:                       True   LL-Null:                       -9.7525
Covariance Type:            nonrobust   LLR p-value:                    0.2526
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Cooperativeness             0.0374      0.719      0.052      0.959      -1.372       1.447
Aggressiveness             -0.5453      0.762     -0.716      0.474      -2.039       0.948
